In [ ]:
%pip install nltk
%pip install torch
%pip install torchaudio
%pip install transformers
%pip install scipy

In [ ]:
tts_model = 'suno/bark' # @param {type:"string"}
classification_model = "nateraw/bert-base-uncased-emotion" # @param {type:"string"}
text = 'We came here to get away from it all, so that our love might blossom. Look at my beloved. The flies have made her ugly and rotten. I wish she were beautiful, like the squirrels of the wood. And so I will cover her in the squirrels of the wood.' # @param {type:"string"}

## Audio outputs:

1. Raw TTS
2. TTS after applying emotion labels

In [ ]:
import nltk
from transformers import pipeline
from IPython.display import Audio, display
import scipy

sound_file_raw = "bark_out_raw.wav"
sound_file_emo = "bark_out_emo.wav"

classify = pipeline("text-classification", model=classification_model)
tts = pipeline("text-to-speech", tts_model)

sentences = list(nltk.sent_tokenize(text))
labels = []

for sentence in sentences:
    classification = classify(sentence)[0]
    labels.append(f'[{classification["label"]}]')
    print(f"{classification['label']}: {sentence}")

# Flatten the list of tuples
classified_sentences = list(sum(zip(labels, sentences), ()))
text_with_labels = " ".join(classified_sentences)

print('Generation voice without labels:', text)
speech = tts(text, forward_params={"do_sample": True})
scipy.io.wavfile.write(sound_file_raw, rate=speech["sampling_rate"], data=speech["audio"])

print('Generation voice with labels:', text_with_labels)
speech = tts(text_with_labels, forward_params={"do_sample": True})
scipy.io.wavfile.write(sound_file_emo, rate=speech["sampling_rate"], data=speech["audio"])

display(Audio(sound_file_raw, autoplay=False))
display(Audio(sound_file_emo, autoplay=False))